In [151]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

Import the data from the webpage and save it as a new Beautiful Soup object _soup_

In [152]:
path = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(path)
assert(result.status_code == 200) # make sure the request came back positive from the server
soup = BeautifulSoup(result.content, 'html5lib')

In [153]:
table = soup.body.table
df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighbourhood'])

rows = table.findAll('tr')
for i, row in enumerate(rows):
    values = [x.getText().rstrip('\n') for x in row.findAll('td')]
    if len(values) > 0 : df.loc[i] = values

print(df.shape)
df.head()

(289, 3)


,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


Extract the DataFrame with postcodes from the resulting list _wiki_, do some formatting and remove '_Not assigned_' values from the Borough column

In [154]:
df = df[df['Borough'] != 'Borough']
df = df[df['Borough'] != 'Not assigned']

df.reset_index(drop=True, inplace=True)

print(df.shape)
df.head(10)

(212, 3)


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Now set the Neighbourhood to the Borough when its value it 'Not assigned'

In [155]:
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [156]:
codes = pd.DataFrame(df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)))
codes.reset_index(inplace=True) # Grouping the DataFrame made the columns Postcode and Borough the indices
codes.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [157]:
codes.shape

(103, 3)